In [1]:
import os

In [2]:
%pwd

'd:\\Deepak_Work\\AI-Projects\\AI-Projects\\Text Summarizer Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Deepak_Work\\AI-Projects\\AI-Projects\\Text Summarizer Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
import importlib
import TextSummarizer.components as components
importlib.reload(components)

from TextSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
print(CONFIG_FILE_PATH)
print(PARAMS_FILE_PATH)

config\config.yaml
params.yaml


In [8]:
import TextSummarizer.components
print(TextSummarizer.components.__file__)


D:\Deepak_Work\AI-Projects\AI-Projects\Text Summarizer Project\src\TextSummarizer\components\__init__.py


In [9]:
with open(TextSummarizer.components.__file__, 'r') as f:
    print(f.read())


In [10]:
CONFIG_FILE_PATH = Path("config/config.yaml")


In [11]:
from TextSummarizer.constants import CONFIG_FILE_PATH
from TextSummarizer.constants import PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH): 

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
 
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]

        create_directories([config["root_dir"]])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config["root_dir"]),
            source_URL=config["source_URL"],
            local_data_file=Path(config["local_data_file"]),
            unzip_dir=Path(config["unzip_dir"]),
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
import TextSummarizer.logging as logger
from TextSummarizer.utils.common import get_size

In [13]:
# rerunning our ingestion, delete the old bad file so it can redownload properly
import os
bad_zip = "artifacts/data_ingestion/data.zip"
if os.path.exists(bad_zip):
    os.remove(bad_zip)
    print("🗑️ Deleted old invalid ZIP file.")

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> Path:
        logger.info("Starting data download...")
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(
                f"Data downloaded successfully! and is saved at {self.config.local_data_file} "
                f"and the file size is {get_size(Path(filename))}"
            )
        else:
            logger.info(f"File already exists of size {get_size(self.config.local_data_file)}")

        return self.config.local_data_file

    def extract_zip_file(self, zip_file_path: Path, unzip_dir: Path) -> None:
        logger.info("Starting data extraction...")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
        logger.info(f"Data extracted successfully at {unzip_dir}")

In [15]:
from TextSummarizer.logging import logger

try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    zip_file_path = data_ingestion.download_data()

    data_ingestion.extract_zip_file(
        zip_file_path=zip_file_path,
        unzip_dir=data_ingestion_config.unzip_dir,
    )

except Exception as e:
    logger.exception(e)  # ✅ works fine now
    raise e


[2025-11-01 17:18:24,977: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2025-11-01 17:18:24,980: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-11-01 17:18:24,980: INFO: common: created directory at: artifacts]
[2025-11-01 17:18:24,984: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-01 17:18:24,984: INFO: 3790782038: Starting data download...]
[2025-11-01 17:18:28,623: INFO: 3790782038: Data downloaded successfully! and is saved at artifacts\data_ingestion\data.zip and the file size is ~ 8073 KB]
[2025-11-01 17:18:28,623: INFO: 3790782038: Starting data extraction...]
[2025-11-01 17:18:28,849: INFO: 3790782038: Data extracted successfully at artifacts\data_ingestion]
